# カスミ 小霞

## sp計算方式 - 未進化 L.v1~9

所有參數

In [1]:
import sqlite3
import pandas as pd
import datetime

class SamePokemon():
    def __init__(self):
        self.conn = sqlite3.connect("../pokemon_database.db")
        self.cursor = self.conn.cursor()
        self.cursor.execute('SELECT * FROM pokemon')
        self.data = self.cursor.fetchall()

    def get_data_as_dataframe(self):
        columns = [
            "id",
            "img_num",
            "name",
            "sp",
            "expertise",
            "level",
            "help_fruit",
            "help_fruit_num",
            "help_ingredient_1",
            "help_ingredient_num_1",
            "help_ingredient_2",
            "help_ingredient_num_2",
            "help_ingredient_3",
            "help_ingredient_num_3",
            "help_time",
            "help_max",
            "skill_main",
            "skill_main_num",
            "skill_main_level",
            "skill_second_1",
            "skill_second_num_1",
            "skill_second_2",
            "skill_second_num_2",
            "skill_second_3",
            "skill_second_num_3",
            "skill_second_4",
            "skill_second_num_4",
            "skill_second_5",
            "skill_second_num_5",
            "power_up",
            "power_down",
            "備註"
        ]

        df = pd.DataFrame(self.data, columns=columns)
        return df

# 創建一個物件
pokemon_instance = SamePokemon()

# 將資料以 DataFrame 格式讀取
pokemon_data = pokemon_instance.get_data_as_dataframe()

# 取得時間字串
time_strings = pokemon_data['help_time']

def convert_to_time_format(time_str):
    # 將時間字串轉換為時間物件
    time_obj = datetime.datetime.strptime(time_str, '%H:%M:%S').time()
    # 將時間物件轉換為整數，以時 * 10000 + 分 * 100 + 秒格式表示
    return time_obj.hour * 10000 + time_obj.minute * 100 + time_obj.second

# 將時間字串轉換為整數格式
time_converted = [convert_to_time_format(time_str) for time_str in time_strings]

# 更新 'help_time' 欄位
time_indices =list(range(2, 5))+ list(range(34, 36))+ list(range(48, 49))+ list(range(54, 55))+ list(range(62, 64))+ list(range(75, 76))+ list(range(77, 78))

for i, idx in enumerate(time_indices):
    pokemon_data.at[idx, 'help_time'] = time_converted[i]


In [2]:
# 取y值
y = pokemon_data.iloc[list(range(2, 5))+ list(range(34, 36))+ list(range(48, 49))+ list(range(54, 55))+ list(range(62, 64))+ list(range(75, 76))+ list(range(77, 78)), 3]

In [3]:
#取X值
X = pokemon_data.iloc[list(range(2, 5))+ list(range(34, 36))+ list(range(48, 49))+ list(range(54, 55))+ list(range(62, 64))+ list(range(75, 76))+ list(range(77, 78)), [5, 7, 9] + list(range(14, 19))+ list(range(29,31))]

In [4]:
# 對 skill_main 進行 Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X['skill_main_encoded'] = label_encoder.fit_transform(X['skill_main'])
# 除去原始的 skill_main 欄位
X = X.drop('skill_main', axis=1)


# 繼續對其他類別型欄位做 One-Hot Encoding
categorical_columns = ['power_up', 'power_down']

for col in categorical_columns:
    X = pd.concat([X, pd.get_dummies(X[col], prefix=col, drop_first=True)], axis=1)
    X = X.drop(col, axis=1)

## 把數據分割成訓練集&測試集

In [5]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=2)
# test_size=0.3 = 隨意抓取30%測試集
# random_state 重跑訓練值幾遍

In [6]:
from sklearn.linear_model import LinearRegression 
regressor=LinearRegression() # 初始化
model=regressor.fit(X_train,y_train)

In [7]:
y_predict=regressor.predict(X_test)

from sklearn.metrics import r2_score # 2的平方越大(越接近1)，多元線性回歸就越好
score1=r2_score(y_test,y_predict) # 比較訓練集&測試集之間的預測好不好

In [8]:
score1

0.5986905399195248

In [9]:
model.coef_ # y=b+7.03471165e*data1+1.20332163e*data2+3.96425367e*data3+1.53936312e*data4 -4.80880533e*data5

array([ 9.19983161e+00,  2.06531437e+01, -8.34808894e+00,  3.25987235e-03,
        3.53510870e+01,  1.66525783e-02,  0.00000000e+00,  9.99746114e+00,
        0.00000000e+00,  7.97629526e-01,  4.77725906e+00,  3.62494303e+00,
       -8.40220209e+00,  3.62494303e+00,  4.77725906e+00])

In [10]:
model.intercept_ #b=74.48195300240218

-111.96407567296416

In [11]:
model.score(X,y) # 再次評估loss值 =>跟score1比較=>數據可用

0.8501064106067299